In [ ]:
import math
import random
import re
from collections import Counter, defaultdict

import numpy as np
import matplotlib.pyplot as plt

try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    TORCH_AVAILABLE = True
except ModuleNotFoundError:
    torch = None
    nn = None
    optim = None
    TORCH_AVAILABLE = False


# 事前学習（Pre-training）

事前学習は、大量の未ラベルテキストから「次にどのトークンが来るか」を予測することで、言語の統計構造を学ぶ段階です。
このノートでは、データ準備、N-gramからニューラル言語モデルへの流れ、評価（perplexity）、計算コスト感覚、そして機械論的解釈可能性の入口までを一貫して確認します。

言語モデルの基本目的は次の確率を最大化することです。

`P(w_1, ..., w_T) = Π_t P(w_t | w_{<t})`

ここで `w_{<t}` は `w_1 ... w_{t-1}`（時刻 `t` より前のトークン列）です。
実装ではクロスエントロピー損失を最小化します。
評価では perplexity（困惑度）を使い、`perplexity = exp(平均クロスエントロピー)`、低いほど予測が当たりやすいことを意味します。


In [ ]:
# 小さな日本語コーパス（教育用）
raw_docs = [
    '事前学習では大量のテキストから次トークン予測を学ぶ。',
    '言語モデルは文脈に応じた分布を出力する。',
    'トークン化の設計は学習効率と性能に強く影響する。',
    '前処理では重複除去や品質フィルタリングが重要になる。',
    '評価ではperplexityや下流タスク性能を併用する。',
    'モデル規模とデータ規模のバランスが学習の成否を左右する。',
    '機械論的解釈可能性は内部回路の理解に役立つ。',
    'SFTは事前学習済みモデルを指示追従に調整する工程である。',
    '推論時は温度やサンプリング戦略が出力を変える。',
    '長文文脈では注意機構の設計が効いてくる。',
]


def normalize(s):
    s = s.lower().strip()
    s = re.sub(r'[。､，,.!?！？]', ' ', s)
    s = re.sub(r'\s+', ' ', s)
    return s


def char_tokenize(s):
    s = normalize(s).replace(' ', '')
    return list(s)


docs = [char_tokenize(d) for d in raw_docs]
random.seed(0)
random.shuffle(docs)
split = int(len(docs) * 0.8)
train_docs = docs[:split]
val_docs = docs[split:]

print('train docs:', len(train_docs), 'val docs:', len(val_docs))
print('sample train tokens:', ''.join(train_docs[0][:30]))


In [ ]:
train_lengths = np.array([len(d) for d in train_docs], dtype=np.int64)
val_lengths = np.array([len(d) for d in val_docs], dtype=np.int64)

vocab = sorted(set(ch for doc in train_docs for ch in doc))
vocab = ['<unk>'] + vocab
stoi = {ch: i for i, ch in enumerate(vocab)}
itos = {i: ch for ch, i in stoi.items()}
unk_id = stoi['<unk>']

print('vocab size:', len(vocab))
print('avg train length:', float(train_lengths.mean()))
print('avg val length  :', float(val_lengths.mean()))

plt.figure(figsize=(6.8, 3.4))
plt.hist(train_lengths, alpha=0.7, label='train', color='#7aa2ff')
plt.hist(val_lengths, alpha=0.7, label='val', color='#8dd3a7')
plt.xlabel('sequence length (characters)')
plt.ylabel('count')
plt.title('Length distribution after tokenization')
plt.legend()
plt.tight_layout()
plt.show()


まずはN-gram言語モデルを作り、事前学習の最小ベースラインを確認します。

- Uni-gram: 直前文脈を使わず出現頻度だけで予測
- Bi-gram: 1つ前のトークンを条件に次トークンを予測

ここでは加法スムージング（add-k）を入れて、未知遷移でも確率0を避けます。
また簡略化のためBOS/EOSは入れず、生成時の開始文字を固定しています（実運用ではBOS開始・EOS停止が一般的）。


In [ ]:
def flatten(docs):
    out = []
    for d in docs:
        out.extend(d)
    return out


def to_ids(doc):
    return [stoi.get(ch, unk_id) for ch in doc]


train_ids = [to_ids(d) for d in train_docs]
val_ids = [to_ids(d) for d in val_docs]

unigram_counts = Counter(flatten(train_ids))
bigram_counts = defaultdict(Counter)
for seq in train_ids:
    for a, b in zip(seq[:-1], seq[1:]):
        bigram_counts[a][b] += 1

V = len(vocab)


def unigram_prob(tok, k=0.1):
    total = sum(unigram_counts.values())
    return (unigram_counts[tok] + k) / (total + k * V)


def bigram_prob(prev, tok, k=0.1):
    row = bigram_counts[prev]
    total = sum(row.values())
    return (row[tok] + k) / (total + k * V)


def perplexity_unigram(seqs):
    nll = 0.0
    n = 0
    for seq in seqs:
        for t in seq:
            p = unigram_prob(t)
            nll += -math.log(p + 1e-12)
            n += 1
    return math.exp(nll / max(n, 1))


def perplexity_bigram(seqs):
    nll = 0.0
    n = 0
    for seq in seqs:
        for prev, t in zip(seq[:-1], seq[1:]):
            p = bigram_prob(prev, t)
            nll += -math.log(p + 1e-12)
            n += 1
    return math.exp(nll / max(n, 1))


print('val perplexity unigram:', round(perplexity_unigram(val_ids), 4))
print('val perplexity bigram :', round(perplexity_bigram(val_ids), 4))


In [ ]:
def sample_unigram(max_len=40):
    probs = np.array([unigram_prob(i) for i in range(V)], dtype=np.float64)
    probs = probs / probs.sum()
    ids = np.random.choice(np.arange(V), size=max_len, p=probs)
    return ''.join(itos[i] for i in ids if i != unk_id)


def sample_bigram(start_id, max_len=40):
    out = [start_id]
    cur = start_id
    for _ in range(max_len - 1):
        probs = np.array([bigram_prob(cur, j) for j in range(V)], dtype=np.float64)
        probs = probs / probs.sum()
        nxt = int(np.random.choice(np.arange(V), p=probs))
        out.append(nxt)
        cur = nxt
    return ''.join(itos[i] for i in out if i != unk_id)


np.random.seed(1)
start_char = train_ids[0][0]
print('unigram sample:', sample_unigram(36))
print('bigram sample :', sample_bigram(start_char, 36))


次に、クロスエントロピー損失を手計算し、ニューラル言語モデル学習へ接続します。

In [ ]:
logits = np.array([0.2, -0.3, 1.1, 0.0], dtype=np.float64)
target = 2

shift = logits - np.max(logits)
probs = np.exp(shift) / np.sum(np.exp(shift))
ce = -math.log(probs[target] + 1e-12)

print('probs =', np.round(probs, 4))
print('cross entropy =', round(float(ce), 6))
print('perplexity for this token =', round(float(math.exp(ce)), 6))


In [ ]:
if TORCH_AVAILABLE:
    torch.manual_seed(0)

    # 連結テキストを作成
    train_text = ''.join(''.join(d) + '\n' for d in train_docs)
    val_text = ''.join(''.join(d) + '\n' for d in val_docs)

    train_data = torch.tensor([stoi.get(ch, unk_id) for ch in train_text], dtype=torch.long)
    val_data = torch.tensor([stoi.get(ch, unk_id) for ch in val_text], dtype=torch.long)

    max_block = min(48, len(train_data) - 1, len(val_data) - 1)
    if max_block < 2:
        raise ValueError('Dataset too short for neural LM demo after split')

    block = max_block
    batch_size = 32

    def get_batch(data, block_size, bsz):
        high = len(data) - block_size - 1
        idx = torch.randint(0, high + 1, (bsz,))
        x = torch.stack([data[i:i+block_size] for i in idx])
        y = torch.stack([data[i+1:i+block_size+1] for i in idx])
        return x, y

    class TinyPretrainLM(nn.Module):
        def __init__(self, vocab_size, d_model=64):
            super().__init__()
            self.emb = nn.Embedding(vocab_size, d_model)
            self.gru = nn.GRU(d_model, d_model, batch_first=True)
            self.head = nn.Linear(d_model, vocab_size)

        def forward(self, x):
            h = self.emb(x)
            out, _ = self.gru(h)
            return self.head(out)

    model = TinyPretrainLM(len(vocab), d_model=64)
    opt = optim.AdamW(model.parameters(), lr=3e-3)
    criterion_mean = nn.CrossEntropyLoss()
    criterion_sum = nn.CrossEntropyLoss(reduction='sum')

    for step in range(260):
        xb, yb = get_batch(train_data, block, batch_size)
        logits = model(xb)
        loss = criterion_mean(logits.reshape(-1, len(vocab)), yb.reshape(-1))

        opt.zero_grad()
        loss.backward()
        opt.step()

        if step % 65 == 0:
            print(f'step={step:>3d}, train loss={loss.item():.4f}')

    # バッチ1個ではなく、検証系列全体で近似perplexityを計算
    with torch.no_grad():
        total_nll = 0.0
        total_tok = 0
        starts = list(range(0, len(val_data) - 1, block))
        for s in starts:
            b = min(block, len(val_data) - 1 - s)
            if b <= 0:
                continue
            x = val_data[s:s+b].unsqueeze(0)
            y = val_data[s+1:s+b+1].unsqueeze(0)
            logits = model(x)
            nll = criterion_sum(logits.reshape(-1, len(vocab)), y.reshape(-1)).item()
            total_nll += nll
            total_tok += y.numel()

        val_loss = total_nll / max(total_tok, 1)
        val_ppl = math.exp(val_loss)

    print('val avg nll =', round(val_loss, 4), 'val perplexity =', round(val_ppl, 4))

    # 生成
    prompt = '事前学習'
    ids = [stoi.get(ch, unk_id) for ch in prompt]
    x = torch.tensor(ids, dtype=torch.long).unsqueeze(0)
    for _ in range(40):
        logits = model(x)
        next_id = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
        x = torch.cat([x, next_id], dim=1)
    gen = ''.join(itos[i] if i != unk_id else '□' for i in x.squeeze(0).tolist())
    print('generated:', gen)
else:
    print('PyTorch未導入のため、ニューラルLMセルをスキップしました。')


事前学習ではデータ品質が非常に重要です。
重複が多いと実効データ量が減り、汚染データがあると望ましくない振る舞いを学習します。
下では文字3-gramのJaccard類似度で近重複を簡易検出します。

In [ ]:
def shingles(text, k=3):
    if len(text) < k:
        return {text}
    return {text[i:i+k] for i in range(len(text) - k + 1)}


def jaccard(a, b):
    inter = len(a & b)
    union = len(a | b)
    return inter / max(union, 1)


dedup_candidates = [
    'llmの事前学習では大規模テキストを使う',
    'llmの事前学習では大規模なテキストを使う',
    '画像分類ではラベル付きデータで学習する',
]

S = [shingles(s, k=3) for s in dedup_candidates]
for i in range(len(S)):
    for j in range(i + 1, len(S)):
        sim = jaccard(S[i], S[j])
        print(f'sim({i},{j}) = {sim:.4f}')


スケーリング則の細部は設定依存ですが、実務では「おおまかな計算量感覚」を先に持つのが重要です。
ここではデコーダ型Transformer学習の粗い目安として、

`FLOPs ≈ 6 * N_params * N_tokens`

を使って見積もります（係数はモデル形状・実装・ハードウェアで変動します）。


In [ ]:
def estimate_flops(params_billion, tokens_billion):
    n_params = params_billion * 1e9
    n_tokens = tokens_billion * 1e9
    return 6.0 * n_params * n_tokens


settings = [
    (0.1, 20),   # 0.1B params, 20B tokens
    (0.7, 100),  # 0.7B params, 100B tokens
    (7.0, 300),  # 7B params, 300B tokens
]

for p_b, t_b in settings:
    flops = estimate_flops(p_b, t_b)
    print(f'params={p_b:>4.1f}B, tokens={t_b:>4.0f}B -> FLOPs≈{flops:.3e}')

# トークン課金の例（仮定値）
in_tok, out_tok = 1200, 400
price_per_m_in = 0.20
price_per_m_out = 0.80
cost = (in_tok / 1e6) * price_per_m_in + (out_tok / 1e6) * price_per_m_out
print('\nexample inference cost per request (assumed pricing) =', round(cost, 6), 'USD')


事前学習後には、継続事前学習（domain adaptive pretraining）やSFTへ進むことが多いです。
混合データでの学習では、一般コーパスとドメインコーパスの損失バランスを監視し、過学習や忘却を防ぎます。

In [ ]:
# 2種類のデータ損失を重み付きで合成する簡易例
loss_general = np.array([2.20, 2.05, 1.98, 1.95, 1.93])
loss_domain = np.array([2.80, 2.30, 2.00, 1.82, 1.74])
alpha = 0.6  # general側の重み

mixed = alpha * loss_general + (1 - alpha) * loss_domain
for i, (g, d, m) in enumerate(zip(loss_general, loss_domain, mixed), 1):
    print(f'epoch {i}: general={g:.3f}, domain={d:.3f}, mixed={m:.3f}')

plt.figure(figsize=(6.8, 3.5))
plt.plot(loss_general, marker='o', label='general loss')
plt.plot(loss_domain, marker='s', label='domain loss')
plt.plot(mixed, marker='^', label='weighted objective')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Balancing continued pretraining objectives')
plt.legend()
plt.tight_layout()
plt.show()


機械論的解釈可能性（mechanistic interpretability）は、ニューラルモデル内部の回路を理解する試みです。
下の可視化は厳密な回路解析そのものではなく、
「何が次に出やすいか」を統計遷移として読むための入口デモです。

本格的には、注意ヘッドやMLPニューロン活性を直接解析して因果的に検証します。


In [ ]:
# Bi-gram遷移行列を可視化（解釈可能性の最小例）
show_chars = [ch for ch in ['事', '前', '学', '習', 'モ', 'デ', 'ル', '。'] if ch in stoi]
show_ids = [stoi[ch] for ch in show_chars]

M = np.zeros((len(show_ids), len(show_ids)), dtype=np.float64)
for i, src in enumerate(show_ids):
    for j, dst in enumerate(show_ids):
        M[i, j] = bigram_prob(src, dst)

row_sums = M.sum(axis=1, keepdims=True)
M_norm = M / np.maximum(row_sums, 1e-12)

plt.figure(figsize=(6.0, 4.6))
plt.imshow(M_norm, cmap='magma')
plt.colorbar(label='P(next | current)')
plt.xticks(range(len(show_chars)), show_chars)
plt.yticks(range(len(show_chars)), show_chars)
plt.xlabel('next token')
plt.ylabel('current token')
plt.title('Interpretable transition map (toy bigram)')
plt.tight_layout()
plt.show()

for i, src in enumerate(show_chars):
    top = np.argsort(M_norm[i])[::-1][:3]
    cand = [(show_chars[t], float(M_norm[i, t])) for t in top]
    print(src, '->', [(c, round(p, 4)) for c, p in cand])


事前学習では「データ」「目的関数」「計算資源」の3つを同時に設計する必要があります。
最初に小さな実験で挙動を掴み、次に本番規模へスケールする手順を徹底すると、失敗コストを下げやすくなります。